## Querying data from MeteoFrance API

In [19]:
import requests
from io import StringIO
import pandas as pd
import time, matplotlib
from tqdm import tqdm
import numpy as np

In [30]:
#API key (to be obtained from https://portail-api.meteofrance.fr/web/fr/api/DonneesPubliquesClimatologie after sign in and done "Configurer l'API")
#WARNING the Swagger seems to be broken
apiKey = 'eyJ4NXQiOiJZV0kxTTJZNE1qWTNOemsyTkRZeU5XTTRPV014TXpjek1UVmhNbU14T1RSa09ETXlOVEE0Tnc9PSIsImtpZCI6ImdhdGV3YXlfY2VydGlmaWNhdGVfYWxpYXMiLCJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJGbG9yaWFuQmxhbmNAY2FyYm9uLnN1cGVyIiwiYXBwbGljYXRpb24iOnsib3duZXIiOiJGbG9yaWFuQmxhbmMiLCJ0aWVyUXVvdGFUeXBlIjpudWxsLCJ0aWVyIjoiVW5saW1pdGVkIiwibmFtZSI6IkRlZmF1bHRBcHBsaWNhdGlvbiIsImlkIjo2ODY3LCJ1dWlkIjoiYjk0ZWRjM2EtZTcyZC00MWFkLWIwMTEtNmU4MDgyNjY5MjAzIn0sImlzcyI6Imh0dHBzOlwvXC9wb3J0YWlsLWFwaS5tZXRlb2ZyYW5jZS5mcjo0NDNcL29hdXRoMlwvdG9rZW4iLCJ0aWVySW5mbyI6eyI1MFBlck1pbiI6eyJ0aWVyUXVvdGFUeXBlIjoicmVxdWVzdENvdW50IiwiZ3JhcGhRTE1heENvbXBsZXhpdHkiOjAsImdyYXBoUUxNYXhEZXB0aCI6MCwic3RvcE9uUXVvdGFSZWFjaCI6dHJ1ZSwic3Bpa2VBcnJlc3RMaW1pdCI6MCwic3Bpa2VBcnJlc3RVbml0Ijoic2VjIn19LCJrZXl0eXBlIjoiUFJPRFVDVElPTiIsInN1YnNjcmliZWRBUElzIjpbeyJzdWJzY3JpYmVyVGVuYW50RG9tYWluIjoiY2FyYm9uLnN1cGVyIiwibmFtZSI6IkRvbm5lZXNQdWJsaXF1ZXNDbGltYXRvbG9naWUiLCJjb250ZXh0IjoiXC9wdWJsaWNcL0RQQ2xpbVwvdjEiLCJwdWJsaXNoZXIiOiJhZG1pbl9tZiIsInZlcnNpb24iOiJ2MSIsInN1YnNjcmlwdGlvblRpZXIiOiI1MFBlck1pbiJ9XSwiZXhwIjoxNzA0NjUzODAxLCJ0b2tlbl90eXBlIjoiYXBpS2V5IiwiaWF0IjoxNzA0NjUwMjAxLCJqdGkiOiIxYTExYmNkOS0wNGQ0LTQ5M2YtYjlmNC0zMTdiNTg0ZTE5MTQifQ==.jyMcPomwd0QAj3eXfcwf4fWfJxOpwAd0kFUArGDNlDO1_gQkRJTcJS2-seyYNswwu2s9k9zagH1cEr76tbJelPeGCmDSeKjEwt7ZFSUqZHLJaoQ40HzGu_dSG4JhHs_SJsT-JebRp30euHkJeCtgTpHx4SOZ-d6eCQoCffeiijyiusPfDf-Si25BzAsAeFmzlhrqSWz2k2V7BoHfl7H36AGPNVMhvlua82nxWz4K9vYl44cBMZh-S7eGn-ABmHvBUse-CKitBmx_vxLxZtuT7QjQORchru66Izn1GGh-BH7xKzxxll9qQjv3liyMBGwG-MfYKDDPrpKO29-UDZZz7w=='

In [54]:
def collectData(year, locationCode = '31478001'):
    ### collect data for one year at one station
    print("Processing year %i"%year)

    #define the location and start / finish time (WARNING : max 1 yr period)
    params = {
        'id-station': locationCode, #3141001 is Revel 31478001 is St Felix
        'date-deb-periode': '%i-01-01T00:00:00Z'%year,
        'date-fin-periode': '%i-12-31T23:59:00Z'%year,
    }

    headers = {
        'accept': '*/*',
        'apikey': apiKey
    }

    #first requests prepares the payload returns the ID for downloading it as a file (asynchronous API)
    response = requests.get('https://public-api.meteofrance.fr/public/DPClim/v1/commande-station/horaire', params=params, headers=headers)
    payloadID = response.json()["elaboreProduitAvecDemandeResponse"]["return"]
    print("payload ID : ", payloadID)

    #second request to actually collect the data
    params = {
        'id-cmde':  payloadID
    }
    returnCode = 204
    while returnCode == 204:
        response = requests.get('https://public-api.meteofrance.fr/public/DPClim/v1/commande/fichier', params=params, headers=headers)
        returnCode = response.status_code
        time.sleep(2) #multiple pinging of the API until data is ready

    payload = StringIO(response.text)
    responsedf = pd.read_csv(payload, sep=";")
    # #responsedf.to_csv("data_collected.csv")
    return responsedf

In [58]:
#collecting data over multiple years
allDatadf = pd.DataFrame()
for year in range(2007,2024):
    yeardf = collectData(year)
    if allDatadf.empty:
        allDatadf = yeardf
    else:
        frames = [allDatadf, yeardf]
        allDatadf = pd.concat(frames)
allDatadf.to_csv("allData.csv")

Processing year 2007
payload ID :  758573489918
Processing year 2008
payload ID :  758573507304
Processing year 2009
payload ID :  758573525847
Processing year 2010
payload ID :  758573543301
Processing year 2011
payload ID :  758573557581
Processing year 2012
payload ID :  758573571750
Processing year 2013
payload ID :  758573586861
Processing year 2014
payload ID :  758573602740
Processing year 2015
payload ID :  758573617883
Processing year 2016
payload ID :  758573637051
Processing year 2017
payload ID :  758573657639
Processing year 2018
payload ID :  758573675145
Processing year 2019
payload ID :  758573692846
Processing year 2020
payload ID :  758573711650
Processing year 2021
payload ID :  758573728973
Processing year 2022
payload ID :  758573747170
Processing year 2023
payload ID :  758573761458
